## Simulacion de predicciones secuenciales

In [50]:
import numpy as np
import random
import math
import json

In [51]:
class Agent:
  def __init__(self, belief, strategyFunc, scoringRule):
    self.belief = belief
    self.scoringRule = scoringRule
    self.strategyFunc = strategyFunc

  def getBelief(self):
    return self.belief

  def applyStrategy(self, marketState):
    return self.strategyFunc(marketState)
  
  def getScoringRule(self):
    return self.scoringRule
  
class Belief:
  @staticmethod
  def randomBelief():
    return random.uniform(0, 1)

  @staticmethod
  def fixedBelief(belief):
    return belief

  @staticmethod
  def errorBelief(real_probability, delta):
    return real_probability + random.uniform(-delta, delta)

class ScoringRules:
  c = 0.1

  @staticmethod
  def logRule(p, q):
    if ((p == 0 or p == 1) and q != 0 and q != 1):
      return -math.inf
    elif (q == p and (q == 0 or q == 1)):
      return 0
    return round((q * np.log(p) + (1 - q) * np.log((1 - p))), 4)

  @staticmethod
  def brierRule(p, q):
    return -(q * ((1 - p) ** 2) + (1 - q) * (p ** 2))

  @staticmethod
  def quadraticError(p, q):
    return -((p - q) ** 2)

  @staticmethod
  def f(p, q):
    return round((ScoringRules.c * p + (1 - ScoringRules.c) * q), 2)

  @staticmethod
  def getPredictionScore(p_i, q, rule):
    if rule == "log":
      return ScoringRules.logRule(p_i, q)
    elif rule == "brier":
      return ScoringRules.brierRule(p_i, q)
    else:
      return ScoringRules.quadraticError(p_i, q)


In [54]:
def simulate(agents, w, q):
  n = len(agents)

  scores = [0] * n
  predictions = []
  prediction = 0

  for i, agent in enumerate(agents):
    p = agent.getBelief()
    predictions.append(p)
    prediction += p * w[i]

  marketPrediction = 0
  for i in range(n):
    marketPrediction += w[i] * predictions[i]

  marketProbability = ScoringRules.f(marketPrediction, q)

  for i, agent in enumerate(agents):
    scores[i] = ScoringRules.getPredictionScore(predictions[i], marketProbability, agent.getScoringRule())

  return scores, predictions, marketProbability, marketPrediction


def evaluateStrategies(agents, q, rule):
  n = len(agents)
  w = [1/n] * n

  scores, predictions, marketProbability, marketPrediction = simulate(agents, w, q)

  marketPrediction = round(marketPrediction, 2)
  marketScore = ScoringRules.getPredictionScore(marketPrediction, marketProbability, rule)

  results = {
    "scores": scores,
    "predictions": predictions,
    "marketProbability": marketProbability,
    "marketPrediction": marketPrediction,
    "marketScore": marketScore,
  }

  return results

In [55]:
q = 0.5

agents = [
  Agent(Belief.errorBelief(q, 0.1), None, "brier"),
  Agent(Belief.errorBelief(q, 0.1), None, "brier"),
  Agent(Belief.errorBelief(q, 0.1), None, "brier"),
  Agent(Belief.errorBelief(q, 0.1), None, "brier"),
  Agent(Belief.errorBelief(q, 0.1), None, "brier")
]

results = evaluateStrategies(agents, q, rule="brier")

print(json.dumps(results, indent=2))

{
  "scores": [
    -0.2504044298889227,
    -0.2552444084969715,
    -0.25257846037165765,
    -0.25094151510360513,
    -0.25711904891753573
  ],
  "predictions": [
    0.5201104422856064,
    0.5724182884150925,
    0.4492214575666294,
    0.4693158818995048,
    0.4156255434533903
  ],
  "marketProbability": 0.5,
  "marketPrediction": 0.49,
  "marketScore": -0.2501
}
